# Phase Hamiltonian Mapper I

## Use Sympy to map objective functions into Hamiltonians consisting of pauli matrices.

In [1]:
# Import Sympy and all it libraries
from sympy import *

### Using Sympy we have to define certain structure in order to get the Hamiltonian mapped correctly
### The objective function should be defined with a variable list and the function itself
### For example:

In [2]:
expr = '(x_l*x_m)*(x_i*x_j)'
var = ['x_i', 'x_j', 'x_l','x_m']
sympify(expr)

x_i*x_j*x_l*x_m

### Firstly we need ot check if the variables in the list matches those in given  in  the objective function:

In [3]:
for v in var:
            if(v in expr):
                pass
            else:
                raise ValueError('Variables Mismatch! Unable to find {} in the Objective Function: {}'.format(v, expr))

### Then we need to define the Pauli matrices and the Identity Matrices based on the number variables there are:

In [4]:
# init function
I = symbols('I')
Z = []
X = []
Y = []

# Next we define the pauli Z:
for i in range(len(var)):
    Z.append(symbols('Z_{}'.format(var[i][len(var[i])-1])))
    Y.append(symbols('Y_{}'.format(var[i][len(var[i])-1])))
    X.append(symbols('Y_{}'.format(var[i][len(var[i])-1])))

### Since for each x term we substitute it with 1/2(I - Z) term but first we need to convert the expression into a sympy

In [5]:
sym_expr = sympify(expr)

for i in range(len(var)):
    s_term = (1/2)*(I - Z[i])
    sym_expr = sym_expr.subs(var[i],s_term)
    

sym_expr = expand(sym_expr)
sym_expr

0.0625*I**4 - 0.0625*I**3*Z_i - 0.0625*I**3*Z_j - 0.0625*I**3*Z_l - 0.0625*I**3*Z_m + 0.0625*I**2*Z_i*Z_j + 0.0625*I**2*Z_i*Z_l + 0.0625*I**2*Z_i*Z_m + 0.0625*I**2*Z_j*Z_l + 0.0625*I**2*Z_j*Z_m + 0.0625*I**2*Z_l*Z_m - 0.0625*I*Z_i*Z_j*Z_l - 0.0625*I*Z_i*Z_j*Z_m - 0.0625*I*Z_i*Z_l*Z_m - 0.0625*I*Z_j*Z_l*Z_m + 0.0625*Z_i*Z_j*Z_l*Z_m

### Now we have to simpify the expressions by 
### 1. Multiplcation with the Identity Matrix
### 2. Remove any global factors/ Identity Matrix

In [6]:
sym_expr = sym_expr.subs(I,1)
coe = sym_expr.as_coefficients_dict()

g_phase = coe.get(1) # remove the global phase/values without Z terms
sym_expr = sym_expr - g_phase
sym_expr

0.0625*Z_i*Z_j*Z_l*Z_m - 0.0625*Z_i*Z_j*Z_l - 0.0625*Z_i*Z_j*Z_m + 0.0625*Z_i*Z_j - 0.0625*Z_i*Z_l*Z_m + 0.0625*Z_i*Z_l + 0.0625*Z_i*Z_m - 0.0625*Z_i - 0.0625*Z_j*Z_l*Z_m + 0.0625*Z_j*Z_l + 0.0625*Z_j*Z_m - 0.0625*Z_j + 0.0625*Z_l*Z_m - 0.0625*Z_l - 0.0625*Z_m

In [7]:
p_expr = Poly(sym_expr,Z)
qc_list = p_expr.monoms()
qc_list

[(1, 1, 1, 1),
 (1, 1, 1, 0),
 (1, 1, 0, 1),
 (1, 1, 0, 0),
 (1, 0, 1, 1),
 (1, 0, 1, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1)]

###

In [9]:
import P_Hamil as ph